In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
import math

In [11]:
torch.manual_seed(42)
class dataset(Dataset):
    def __init__(self, size):
        self.means = torch.tensor([[0, 0], [2, 2], [-2, 2]], dtype=torch.float32)
        self.sigma = math.sqrt(2)
        self.len = size
        
    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        z1 = torch.normal(self.means[0], self.sigma)
        z2 = torch.normal(self.means[1], self.sigma)
        z3 = torch.normal(self.means[2], self.sigma)
        return z1
#         return 1/math.sqrt(3)*(z1+z2+z3)

In [12]:
samples = dataset(10000)

In [13]:
samples_loader = DataLoader(samples, batch_size = 50, shuffle = True)
total_mean = torch.tensor([0, 0], dtype = torch.float32)
for batch in samples_loader:
    total_mean[0] += batch[:,0].mean()
    total_mean[1] += batch[:,1].mean()
total_mean = total_mean / len(samples_loader)
total_mean

tensor([-0.0281, -0.0079])

In [14]:
# Generator network
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )

    def forward(self, z):
        return self.model(z)

# Discriminator network
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


In [15]:
latent_dim = 10
# Build the networks
generator = Generator(latent_dim)
discriminator = Discriminator()

# Loss function and optimizers
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=0.001)
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.001)

# Training the GAN
epochs = 100
batch_size = 50
losses_d = []
losses_g = []

for epoch in range(epochs):
    batch = 1
    running_d = 0.0
    running_g = 0.0
    for sample in samples_loader:
        # Generate real and fake samples
        real_samples = sample
        fake_samples = generator(torch.randn(batch_size, latent_dim))

        # Labels for real and fake samples
        real_labels = torch.ones((batch_size, 1))
        fake_labels = torch.zeros((batch_size, 1))

        # Train discriminator
        optimizer_D.zero_grad()
        d_loss_real = criterion(discriminator(real_samples), real_labels)
        d_loss_fake = criterion(discriminator(fake_samples.detach()), fake_labels)
        d_loss = d_loss_real + d_loss_fake
        d_loss.backward()
        optimizer_D.step()

        # Train generator
        optimizer_G.zero_grad()
        g_loss = criterion(discriminator(fake_samples), real_labels)
        g_loss.backward()
        optimizer_G.step()

        # Print progress
        print(f"Epoch {epoch}, batch {batch} D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")
        running_d += d_loss.item()
        running_g += g_loss.item()
        batch += 1
    losses_d.append(running_d / len(samples_loader))
    losses_g.append(running_g / len(samples_loader))



Epoch 0, batch 1 D Loss: 1.4222370386123657, G Loss: 0.5943178534507751
Epoch 0, batch 2 D Loss: 1.4063389301300049, G Loss: 0.5947249531745911
Epoch 0, batch 3 D Loss: 1.3891561031341553, G Loss: 0.5956199765205383
Epoch 0, batch 4 D Loss: 1.3737552165985107, G Loss: 0.5966014266014099
Epoch 0, batch 5 D Loss: 1.364165186882019, G Loss: 0.5998711585998535
Epoch 0, batch 6 D Loss: 1.3825154304504395, G Loss: 0.5996529459953308
Epoch 0, batch 7 D Loss: 1.3460848331451416, G Loss: 0.6038751602172852
Epoch 0, batch 8 D Loss: 1.3459309339523315, G Loss: 0.6027817726135254
Epoch 0, batch 9 D Loss: 1.3410310745239258, G Loss: 0.6067225933074951
Epoch 0, batch 10 D Loss: 1.3315165042877197, G Loss: 0.6071737408638
Epoch 0, batch 11 D Loss: 1.320600986480713, G Loss: 0.607138991355896
Epoch 0, batch 12 D Loss: 1.3185486793518066, G Loss: 0.612561047077179
Epoch 0, batch 13 D Loss: 1.3012454509735107, G Loss: 0.6146442890167236
Epoch 0, batch 14 D Loss: 1.31135094165802, G Loss: 0.6120714545249

Epoch 1, batch 11 D Loss: 1.5922552347183228, G Loss: 1.3772639036178589
Epoch 1, batch 12 D Loss: 1.842559576034546, G Loss: 1.5051072835922241
Epoch 1, batch 13 D Loss: 1.69477117061615, G Loss: 1.2659727334976196
Epoch 1, batch 14 D Loss: 1.4434890747070312, G Loss: 1.4823522567749023
Epoch 1, batch 15 D Loss: 1.6111857891082764, G Loss: 1.2415906190872192
Epoch 1, batch 16 D Loss: 1.2731268405914307, G Loss: 1.5482062101364136
Epoch 1, batch 17 D Loss: 1.5146435499191284, G Loss: 1.3890471458435059
Epoch 1, batch 18 D Loss: 1.325931191444397, G Loss: 1.4241340160369873
Epoch 1, batch 19 D Loss: 1.3779809474945068, G Loss: 1.3611210584640503
Epoch 1, batch 20 D Loss: 1.3989458084106445, G Loss: 1.4332183599472046
Epoch 1, batch 21 D Loss: 1.7433891296386719, G Loss: 1.4942272901535034
Epoch 1, batch 22 D Loss: 1.4886126518249512, G Loss: 1.4827293157577515
Epoch 1, batch 23 D Loss: 1.4972665309906006, G Loss: 1.576045274734497
Epoch 1, batch 24 D Loss: 1.4192047119140625, G Loss: 1.

Epoch 2, batch 36 D Loss: 1.2151119709014893, G Loss: 0.686694324016571
Epoch 2, batch 37 D Loss: 1.3698101043701172, G Loss: 0.6624295711517334
Epoch 2, batch 38 D Loss: 1.4191926717758179, G Loss: 0.7380397319793701
Epoch 2, batch 39 D Loss: 1.2690246105194092, G Loss: 0.6811984181404114
Epoch 2, batch 40 D Loss: 1.172500491142273, G Loss: 0.7505702376365662
Epoch 2, batch 41 D Loss: 1.2341904640197754, G Loss: 0.7218788266181946
Epoch 2, batch 42 D Loss: 1.36381995677948, G Loss: 0.7000271677970886
Epoch 2, batch 43 D Loss: 1.2128262519836426, G Loss: 0.7273072004318237
Epoch 2, batch 44 D Loss: 1.338676929473877, G Loss: 0.6995827555656433
Epoch 2, batch 45 D Loss: 1.267277717590332, G Loss: 0.711185097694397
Epoch 2, batch 46 D Loss: 1.4794107675552368, G Loss: 0.6700713634490967
Epoch 2, batch 47 D Loss: 1.3296923637390137, G Loss: 0.6644529104232788
Epoch 2, batch 48 D Loss: 1.29657781124115, G Loss: 0.7411485910415649
Epoch 2, batch 49 D Loss: 1.3345410823822021, G Loss: 0.6695

Epoch 3, batch 53 D Loss: 1.1767258644104004, G Loss: 1.0569778680801392
Epoch 3, batch 54 D Loss: 1.05368173122406, G Loss: 1.0052599906921387
Epoch 3, batch 55 D Loss: 1.194150447845459, G Loss: 1.0362308025360107
Epoch 3, batch 56 D Loss: 1.1398677825927734, G Loss: 1.0914109945297241
Epoch 3, batch 57 D Loss: 1.118438959121704, G Loss: 1.126818299293518
Epoch 3, batch 58 D Loss: 1.1659252643585205, G Loss: 1.0276870727539062
Epoch 3, batch 59 D Loss: 1.1120171546936035, G Loss: 0.9986006021499634
Epoch 3, batch 60 D Loss: 1.0271179676055908, G Loss: 1.0858086347579956
Epoch 3, batch 61 D Loss: 1.0715782642364502, G Loss: 1.0577845573425293
Epoch 3, batch 62 D Loss: 1.089979887008667, G Loss: 1.0707486867904663
Epoch 3, batch 63 D Loss: 1.0295199155807495, G Loss: 1.1035981178283691
Epoch 3, batch 64 D Loss: 1.1359909772872925, G Loss: 1.0685997009277344
Epoch 3, batch 65 D Loss: 1.1584117412567139, G Loss: 1.0707017183303833
Epoch 3, batch 66 D Loss: 1.1377002000808716, G Loss: 1.0

Epoch 4, batch 73 D Loss: 1.6257396936416626, G Loss: 1.2578221559524536
Epoch 4, batch 74 D Loss: 1.41579008102417, G Loss: 1.247704029083252
Epoch 4, batch 75 D Loss: 1.5904974937438965, G Loss: 1.3103238344192505
Epoch 4, batch 76 D Loss: 1.3748732805252075, G Loss: 1.302679419517517
Epoch 4, batch 77 D Loss: 1.551226019859314, G Loss: 1.349021553993225
Epoch 4, batch 78 D Loss: 1.4476397037506104, G Loss: 1.2729756832122803
Epoch 4, batch 79 D Loss: 1.5798227787017822, G Loss: 1.308732032775879
Epoch 4, batch 80 D Loss: 1.4398823976516724, G Loss: 1.2731295824050903
Epoch 4, batch 81 D Loss: 1.4319634437561035, G Loss: 1.3005328178405762
Epoch 4, batch 82 D Loss: 1.395586371421814, G Loss: 1.3017680644989014
Epoch 4, batch 83 D Loss: 1.5686540603637695, G Loss: 1.2147547006607056
Epoch 4, batch 84 D Loss: 1.486146092414856, G Loss: 1.2946336269378662
Epoch 4, batch 85 D Loss: 1.5414409637451172, G Loss: 1.1938064098358154
Epoch 4, batch 86 D Loss: 1.5817172527313232, G Loss: 1.1943

...

Epoch 94, batch 186 D Loss: 1.4064435958862305, G Loss: 0.6755330562591553
Epoch 94, batch 187 D Loss: 1.3824758529663086, G Loss: 0.7015144228935242
Epoch 94, batch 188 D Loss: 1.3877711296081543, G Loss: 0.6968234777450562
Epoch 94, batch 189 D Loss: 1.4067225456237793, G Loss: 0.6769131422042847
Epoch 94, batch 190 D Loss: 1.3814032077789307, G Loss: 0.6991149187088013
Epoch 94, batch 191 D Loss: 1.4014028310775757, G Loss: 0.6760286092758179
Epoch 94, batch 192 D Loss: 1.390900731086731, G Loss: 0.6980816125869751
Epoch 94, batch 193 D Loss: 1.3900288343429565, G Loss: 0.687835156917572
Epoch 94, batch 194 D Loss: 1.3807252645492554, G Loss: 0.6986792683601379
Epoch 94, batch 195 D Loss: 1.3999521732330322, G Loss: 0.6947664022445679
Epoch 94, batch 196 D Loss: 1.3881937265396118, G Loss: 0.6906923055648804
Epoch 94, batch 197 D Loss: 1.371675968170166, G Loss: 0.7141554951667786
Epoch 94, batch 198 D Loss: 1.3820888996124268, G Loss: 0.69777512550354
Epoch 94, batch 199 D Loss: 1.

Epoch 95, batch 200 D Loss: 1.384580373764038, G Loss: 0.6978998780250549
Epoch 96, batch 1 D Loss: 1.3869893550872803, G Loss: 0.695924699306488
Epoch 96, batch 2 D Loss: 1.3936400413513184, G Loss: 0.687049150466919
Epoch 96, batch 3 D Loss: 1.38298499584198, G Loss: 0.6991584897041321
Epoch 96, batch 4 D Loss: 1.3843154907226562, G Loss: 0.6946141123771667
Epoch 96, batch 5 D Loss: 1.3845951557159424, G Loss: 0.6962587237358093
Epoch 96, batch 6 D Loss: 1.3839638233184814, G Loss: 0.6971935033798218
Epoch 96, batch 7 D Loss: 1.3844821453094482, G Loss: 0.694273054599762
Epoch 96, batch 8 D Loss: 1.3931018114089966, G Loss: 0.6898439526557922
Epoch 96, batch 9 D Loss: 1.3799591064453125, G Loss: 0.7012348771095276
Epoch 96, batch 10 D Loss: 1.3880751132965088, G Loss: 0.7039029598236084
Epoch 96, batch 11 D Loss: 1.3783645629882812, G Loss: 0.6965903639793396
Epoch 96, batch 12 D Loss: 1.3816981315612793, G Loss: 0.6979640126228333
Epoch 96, batch 13 D Loss: 1.387818455696106, G Loss

Epoch 97, batch 22 D Loss: 1.3956851959228516, G Loss: 0.6834062337875366
Epoch 97, batch 23 D Loss: 1.3910067081451416, G Loss: 0.6813844442367554
Epoch 97, batch 24 D Loss: 1.3958332538604736, G Loss: 0.6848882436752319
Epoch 97, batch 25 D Loss: 1.3793424367904663, G Loss: 0.6918010115623474
Epoch 97, batch 26 D Loss: 1.3750449419021606, G Loss: 0.6951026320457458
Epoch 97, batch 27 D Loss: 1.3877699375152588, G Loss: 0.703754723072052
Epoch 97, batch 28 D Loss: 1.389772891998291, G Loss: 0.6905826330184937
Epoch 97, batch 29 D Loss: 1.3984851837158203, G Loss: 0.6833887696266174
Epoch 97, batch 30 D Loss: 1.3734605312347412, G Loss: 0.7097082734107971
Epoch 97, batch 31 D Loss: 1.39041006565094, G Loss: 0.6949863433837891
Epoch 97, batch 32 D Loss: 1.382181167602539, G Loss: 0.7025525569915771
Epoch 97, batch 33 D Loss: 1.3964991569519043, G Loss: 0.6923753619194031
Epoch 97, batch 34 D Loss: 1.3750518560409546, G Loss: 0.6936387419700623
Epoch 97, batch 35 D Loss: 1.38092017173767

Epoch 98, batch 27 D Loss: 1.3835248947143555, G Loss: 0.7112261056900024
Epoch 98, batch 28 D Loss: 1.414483904838562, G Loss: 0.7016953825950623
Epoch 98, batch 29 D Loss: 1.4239802360534668, G Loss: 0.6919097900390625
Epoch 98, batch 30 D Loss: 1.3768588304519653, G Loss: 0.7179889678955078
Epoch 98, batch 31 D Loss: 1.3869097232818604, G Loss: 0.7096879482269287
Epoch 98, batch 32 D Loss: 1.396236777305603, G Loss: 0.7081292867660522
Epoch 98, batch 33 D Loss: 1.3854944705963135, G Loss: 0.7084892392158508
Epoch 98, batch 34 D Loss: 1.390378475189209, G Loss: 0.7049139142036438
Epoch 98, batch 35 D Loss: 1.394860029220581, G Loss: 0.7048198580741882
Epoch 98, batch 36 D Loss: 1.3899732828140259, G Loss: 0.7045086622238159
Epoch 98, batch 37 D Loss: 1.4040451049804688, G Loss: 0.6955049633979797
Epoch 98, batch 38 D Loss: 1.3891427516937256, G Loss: 0.696498692035675
Epoch 98, batch 39 D Loss: 1.393982172012329, G Loss: 0.696503758430481
Epoch 98, batch 40 D Loss: 1.3714828491210938

Epoch 99, batch 30 D Loss: 1.3824584484100342, G Loss: 0.6926795840263367
Epoch 99, batch 31 D Loss: 1.3700796365737915, G Loss: 0.689717173576355
Epoch 99, batch 32 D Loss: 1.3591136932373047, G Loss: 0.6948782205581665
Epoch 99, batch 33 D Loss: 1.4041032791137695, G Loss: 0.6745498776435852
Epoch 99, batch 34 D Loss: 1.3770332336425781, G Loss: 0.7185576558113098
Epoch 99, batch 35 D Loss: 1.4039262533187866, G Loss: 0.6882462501525879
Epoch 99, batch 36 D Loss: 1.396518349647522, G Loss: 0.6823772192001343
Epoch 99, batch 37 D Loss: 1.3870787620544434, G Loss: 0.6976068019866943
Epoch 99, batch 38 D Loss: 1.400280237197876, G Loss: 0.6805263757705688
Epoch 99, batch 39 D Loss: 1.3884902000427246, G Loss: 0.7001078724861145
Epoch 99, batch 40 D Loss: 1.3978967666625977, G Loss: 0.6861353516578674
Epoch 99, batch 41 D Loss: 1.4037044048309326, G Loss: 0.6690787672996521
Epoch 99, batch 42 D Loss: 1.4299702644348145, G Loss: 0.6790015697479248
Epoch 99, batch 43 D Loss: 1.372991919517

In [ ]:
plt.plot(range(epochs), losses_d)
plt.title('D-Loss vs Epochs')
plt.show()

In [ ]:
plt.plot(range(epochs), losses_g)
plt.title('G-Loss vs Epochs')
plt.show()

In [ ]:
# Generate samples using the trained generator
generated_samples = generator(torch.randn(3000, latent_dim)).detach()
gen_mean = torch.tensor([0, 0], dtype = torch.float32)
gen_mean[0] = generated_samples[:,0].mean()
gen_mean[1] = generated_samples[:,1].mean()

In [ ]:
# Plot the generated samples
plt.hist2d(generated_samples[:, 0], generated_samples[:, 1])
plt.colorbar()
plt.plot(total_mean[0], total_mean[1],label='Real Mean', marker='x', color='red')
plt.plot(gen_mean[0], gen_mean[1],label='Generated Mean', marker='x', color='black')
plt.legend()
plt.show()

In [ ]:
# Create a 2D histogram
hist, x_edges, y_edges = np.histogram2d(generated_samples[:, 0], generated_samples[:, 1], bins=20)

# Get bin centers for x and y
x_centers = (x_edges[:-1] + x_edges[1:]) / 2
y_centers = (y_edges[:-1] + y_edges[1:]) / 2

# Create a meshgrid from bin centers
x_mesh, y_mesh = np.meshgrid(x_centers, y_centers)

# Create a 3D plot
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Plot the 3D surface
ax.plot_surface(x_mesh, y_mesh, hist.T, cmap='viridis')

# Add labels
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Frequency')

# Show the plot
plt.show()